# URL Parameter Analysis

## About

Made by [Jonathan Moore](https://jonathanmoore.digital/). Get in touch via [https://twitter.com/nathanless](https://twitter.com/nathanless) if you have any questions.

## Description

This is a sample workbook to give you some ideas on how to analyse URL parameters.


### Use Cases

These are just a cople of examples:
- Understanding in more detail the types of URL parameters that are present in your data
- Finding URL parameters to remove from datasets, to make it easy to see the wood from the trees
 - For expample Google Analytics (GA)
     - GA has a [table limit of 50,000 rows](https://support.google.com/analytics/answer/1009671?hl=en)
     - And for some reason fbclid is never excluded from reports
- Identifing UTM campgain tracking parameters in internal links
 - This screws up GA, becuase it creates a new session :(
- Spot checking parameters in large datasets. 
 - Good luck doing this in Excel or Google Sheets

## Import packages

In [1]:
import pandas as pd
import numpy as np

## Set variables

Change this to the location of your own file.

In [2]:
file_to_read = 'test-url-parameter-data.csv' 

Set this to matach the column you wish to analysise, for eaxample a Screaming Frog export would be 'address'. 

In [3]:
analysis_dimension = 'landing_page_path'

## Load CSV file

Load CSV file into a new dataframe. Here we're loading some spoof data I created using the [Faker](https://github.com/joke2k/faker) package and some real GA data. I've delebratly used a large file with some time series data but you don't have to.

One enhancement to this script would be to connect it directly to the GA API, if you are interested this [blog by JP Chouinard](https://www.jcchouinard.com/google-analytics-api-using-python/) is a great primer on how to do this using a sercice account.



In [4]:
df = pd.read_csv(file_to_read, sep=',', low_memory=False,  encoding='utf8')

Lowercase all columns and replace spaces with underscore. I find this makes life easier.

In [5]:
df.columns = df.columns.str.lower().str.replace(' ','_')

Create a dynamic URL column to check how many dynamic rows there are

In [6]:
df['dynamic_url'] = df[analysis_dimension].str.contains('\?')

Spot check that csv has loaded as expected. 

In [7]:
print(df.shape[0], 'rows and ', df.shape[1], 'columns\n')
df.head()

1013067 rows and  4 columns



,date,landing_page_path,sessions,dynamic_url
0,2019-06-28,/certain-order-cold,3962,False
1,2019-11-17,/way-while,2775,False
2,2020-12-16,/onto-establish,2562,False
3,2019-06-29,/later-them-eye-next,1599,False
4,2019-11-17,/attorney-talk,1565,False


Check how many pages in the DataFrame are dynamic

In [8]:
dynamic_df = (df.groupby('dynamic_url').agg({analysis_dimension:'count'})
                .rename(columns={analysis_dimension:'frequency'})
                .sort_values('frequency', ascending=False)
                .reset_index())

In [9]:
dynamic_df['%'] = round(dynamic_df['frequency'] / dynamic_df['frequency'].sum(),2)

In [10]:
dynamic_df

,dynamic_url,frequency,%
0,True,614042,0.61
1,False,399025,0.39


## Create a dataframe of all URL parameters and their values

Copy the orginal df, again I fond this makes life easier. Especially when I make a mistake. (Which happens regaually).

In [11]:
parameters = df.copy()

Extract all URL parametees into a Pandas DataFrame using extractall method and named groups to create the DataFrame.

[Stack Overflow Link](https://stackoverflow.com/questions/33288420/extracting-url-parameters-into-pandas-dataframe)

In [12]:
parameters = (parameters[analysis_dimension]
                .str.extractall('[?&](?P<parameter>[^?#=]+)=?(?P<value>[^&]*)')
                .xs(0, level="match")
                .reset_index(drop=True))

Check the new dataframe.

In [13]:
print(parameters.shape[0], 'rows and ', parameters.shape[1], 'columns\n')
parameters.head(5)

121681 rows and  2 columns



,parameter,value
0,size,6_20_7
1,product_list_order,price
2,p,11
3,size,6_20_7
4,size,6_20_7


##### Download to CSV

In [14]:
# parameters.to_csv('url-parameter-values.csv', index=False)

## Sumerise data to see the wood from the trees

Create a new dataframe to summerise parameters and values.

In [15]:
parameters_counts = (parameters.groupby('parameter')
                        .agg({'parameter':'count','value':'nunique'})
                        .rename(columns={'parameter':'frequency','value':'unique_values'})
                        .sort_values('unique_values', ascending=False)
                        .reset_index())

Here a ratio column can be used to identify tracking or internal site searrch parameters. 

The higher ratio the more likely it is that every value is unique. 

In [16]:
parameters_counts['ratio'] = round(parameters_counts['unique_values'] / parameters_counts['frequency'],2)

Check our new dataframe.

In [17]:
print(parameters_counts.shape[0], 'rows and ', parameters_counts.shape[1], 'columns\n')
parameters_counts.iloc[0:10]

26 rows and  4 columns



,parameter,frequency,unique_values,ratio
0,dm_i,107704,91873,0.85
1,mc_eid,273,242,0.89
2,fbclid,221,217,0.98
3,size,3076,156,0.05
4,siteID,5044,141,0.03
5,price,229,38,0.17
6,style,392,28,0.07
7,color_group,106,20,0.19
8,p,3150,12,0.00
9,product_list_limit,536,4,0.01


##### Download to CSV

In [18]:
# parameters_counts.to_csv('url-parameter-analysis.csv', index=False)

## Classify parameters

Classifing parameters can help get a feel of what needs to be handled. 

Here I've used the lists Sitebulb have very kindly shared in their documentation as a starting point:

- https://sitebulb.com/hints/internal/query-string-contains-paginated-parameters/
- https://sitebulb.com/hints/internal/query-string-contains-search-or-filter-parameters/
- https://sitebulb.com/hints/internal/query-string-contains-sort-parameters/

I've also added a list of common tracking parameters. Let me know if you have a more complete list :)

You can adapt the following lists to suit your own requiremnts.

In [19]:
pagination_parameters_list = ['p','page','start','from','limit','index','pageindex','page-index']

internal_site_search_parameters_list = ['r','query','result','fids','fids','refine','search','filter','keyword','q']

sort_parameters_list = ['sby','sort','sortby','orderby','rank','rank-by','option']

tracking_parameters_list = ['mkt_tok','utm_source','utm_medium','utm_campaign','utm_term','utm_content','fbclid','dm_i','dm_t','mc_cid','mc_eid']

Use np.where to act like an Excel if statement to create a new column in the parameters_counts DataFrame.

In [20]:
parameters_counts['parameter_type'] = (np.where(parameters_counts['parameter']
                                        .isin(tracking_parameters_list),'Tracking Parameter',
                                        np.where(parameters_counts['parameter']
                                            .isin(pagination_parameters_list),'Pagination Parameter',
                                        np.where(parameters_counts['parameter']
                                            .isin(internal_site_search_parameters_list),'Intenrnal Site search Parameter',
                                        np.where(parameters_counts['parameter']
                                            .isin(sort_parameters_list),'Sort Parameter',
                                                  'Other')))))

In [21]:
print(parameters_counts.shape[0], 'rows and ', parameters_counts.shape[1], 'columns\n')
parameters_counts.head(10)

26 rows and  5 columns



,parameter,frequency,unique_values,ratio,parameter_type
0,dm_i,107704,91873,0.85,Tracking Parameter
1,mc_eid,273,242,0.89,Tracking Parameter
2,fbclid,221,217,0.98,Tracking Parameter
3,size,3076,156,0.05,Other
4,siteID,5044,141,0.03,Other
5,price,229,38,0.17,Other
6,style,392,28,0.07,Other
7,color_group,106,20,0.19,Other
8,p,3150,12,0.00,Pagination Parameter
9,product_list_limit,536,4,0.01,Other


##### Download to CSV

In [22]:
# parameters_counts.to_csv('url-parameter-analysis.csv', index=False)

Group DataFrame to summerise parameter types.

In [23]:
parameter_types = parameters_counts.copy()

In [24]:
parameter_types = (parameter_types.groupby('parameter_type').agg({'parameter':'count'})
                .rename(columns={'parameter':'frequency'})
                .sort_values('frequency', ascending=False)
                .reset_index())

In [25]:
parameter_types.head()

,parameter_type,frequency
0,Other,18
1,Tracking Parameter,6
2,Intenrnal Site search Parameter,1
3,Pagination Parameter,1


## Spot check parameters

To help you sanity check you can adjust the parameter name in the cell below to get an overview of the first 10 parameters.

This can be a lot easier than trying to fiter larger files in Excel or Google Sheets.

In [26]:
list(set(parameters[parameters['parameter'] == 'dm_i']['value']))[0:10]

['1TPL%2C6NMFG%2CUGE6M3%2CQIY54%2C1',
 '1TPL%2C760AI%2CQJY616%2CT1QR6%2C1',
 '1TPL%2C6LXOX%2CLNJLXD%2CQB2W9%2C1',
 '1TPL%2C6NUF6%2CRS1NVS%2CQK1N1%2C1',
 '1TPL%2C6ISHJ%2CP24WYT%2CPWNC9%2C1',
 '1TPL%2C6LPZ6%2CS6BR72%2CQA38G%2C1',
 '1TPL%2C6WO9O%2CQJWEG2%2CRR6DN%2C1',
 '1TPL%2C6NA61%2COUR5AL%2CQHBXQ%2C1',
 '1TPL%2C6ZV44%2CS6BRLG%2CS6YXO%2C1',
 '1TPL%2C6MRY1%2CCCM3ZH%2CQEYWW%2C1']

## Create a list of query  parameters to exclude in Google Analytics

The next cell turns the parameter column in the parameters_counts dataframe into a string which you can directly input into GA view settings.

Remove or adjust the **[0:10]** slice to suit your needs, but always sanity check, beofre you exclude anything from reports.

When you first upload to GA, it's a good idea to create a test view. Once you're happy copy over the new settings to main reporting veiw.

[Google Analytics help centre link](https://support.google.com/analytics/answer/1010249?hl=en)

In [27]:
ga_string = ', '.join([str(param) for param in list(parameters_counts['parameter'])[0:10]])

ga_string

'dm_i, mc_eid, fbclid, size, siteID, price, style, color_group, p, product_list_limit'

## Cleaning URL parameters

Here we are going to add a column to our original dataframe, which does not contain any parameters.

This first example uses split and take the first index to remove both the query and the fragment.

In [28]:
clean_df = df.copy()

In [29]:
clean_df['clean_url_path'] =  clean_df[analysis_dimension].str.split('?').str[0].str.split('#').str[0]

In [30]:
print(clean_df.shape[0], 'rows and ', clean_df.shape[1], 'columns\n')
clean_df.head()

1013067 rows and  5 columns



,date,landing_page_path,sessions,dynamic_url,clean_url_path
0,2019-06-28,/certain-order-cold,3962,False,/certain-order-cold
1,2019-11-17,/way-while,2775,False,/way-while
2,2020-12-16,/onto-establish,2562,False,/onto-establish
3,2019-06-29,/later-them-eye-next,1599,False,/later-them-eye-next
4,2019-11-17,/attorney-talk,1565,False,/attorney-talk


This second example uses the excellent urllib library.

In [31]:
from urllib.parse import urlsplit, urlunsplit

def remove_query_params_and_fragment(url):
    return urlunsplit(urlsplit(url)._replace(query="", fragment=""))

In [32]:
# clean_df['clean_url_path'] =  clean_df[analysis_dimension].apply(remove_query_params_and_fragment)

Check query string has been removed.

In [33]:
dynamic_df_check = (clean_df[clean_df[analysis_dimension] != clean_df['clean_url_path']]
                    [[analysis_dimension,'clean_url_path']])

In [34]:
print(dynamic_df_check.shape[0], 'rows and ', dynamic_df_check.shape[1], 'columns\n')
dynamic_df_check.head()

614042 rows and  2 columns



,landing_page_path,clean_url_path
3788,/ability-care-space?=p%3D1,/ability-care-space
4816,/attorney-talk?=price%3D0-50,/attorney-talk
9066,/as-media-area?=p%3D1,/as-media-area
9894,/these-draw-while?=___store%3Dview_eu&size=6_2...,/these-draw-while
11769,/machine-among?=dm_t%3D0%2C0%2C0%2C0%2C0,/machine-among


## Remove specic parameters from the original dataframe


In this example, we're going to filter out tracking parameters. Alternatively create your own custom list of parameters to remove. 


Below is my go to list of tracking parameters, which covers:

- Google Abnalytics
- Facebook
- Mailchimp
- Marketo


In [35]:
params_to_remove = [
    'mkt_tok',
    'utm_source',  
    'utm_medium', 
    'utm_campaign',
    'utm_term', 
    'utm_content',
    'fbclid',
    'dm_i',
    'dm_t',
    'mc_cid',
    'mc_eid',
]

But before we do anything let's create a copy of the original df.

In [36]:
filtered_df = df.copy()

[Origninal Script](https://gist.github.com/tyndyll/e254ae3da2d0427371733443152c1337) - I can't take credit for this

In [37]:
from urllib.parse import parse_qs, urlparse, urlencode

def remove_tracker_params(query_string):

    params = []
    for param, values in parse_qs(query_string).items():
        if param not in params_to_remove:
            # value will be a list, extract each one out
            for value in values:
                params.append((param, value))
    return urlencode(params)

def clean_url(url):

    parsed = urlparse(url)
    parsed = parsed._replace(query=remove_tracker_params(parsed.query))
    parsed = parsed._replace(fragment=remove_tracker_params(parsed.fragment))
    return parsed.geturl()

In [38]:
filtered_df[analysis_dimension] = filtered_df[analysis_dimension].apply(clean_url)

##### Download to CSV

In [39]:
# filtered_df.to_csv('filtered-url-parameter-data.csv',index=False)

In [40]:
print('Fin')

Fin
